In [1]:
fla_dir = "/data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla"

In [2]:
from fla_utils import create_temp_file, search
import os
import subprocess
import gzip
import shutil
import nibabel as nib
import numpy as np
from pathlib import Path

def cifti_separate(dscalar):
    data = {}
    data['volume'] = create_temp_file('.nii')
    data['cortex_left'] = create_temp_file('.func.gii')
    data['cortex_right'] = create_temp_file('.func.gii')
    # Separate dsclar
    cmd = f"wb_command -cifti-separate {dscalar} COLUMN -volume-all {data['volume']} -metric CORTEX_LEFT {data['cortex_left']} -metric CORTEX_RIGHT {data['cortex_right']}"
    os.system(cmd)
    for i in data.values():
        assert Path(i).exists()
    # Uncompress format
    for func in [data['cortex_left'], data['cortex_right']]:
        cmd = f"wb_command -gifti-convert BASE64_BINARY {func} {func}"
        os.system(cmd)
    
    return data

def get_surface_vertex_areas(
    surf_left="/data/dump/surfaces/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii", 
    surf_right="/data/dump/surfaces/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii"
):
    for s in [surf_left, surf_right]:
        assert Path(s).exists()
    data = {}
    for hemi, surf_path in zip(
        ['cortex_left','cortex_right'],
        [surf_left,surf_right]
    ):
        data[hemi] = create_temp_file('.func.gii')
        cmd = f"wb_command -surface-vertex-areas {surf_path} {data[hemi]}"
        os.system(cmd)
        assert Path(data[hemi]).exists()
        
    return data

def merge_niftis(niftis):
    merged_nifti = create_temp_file('.nii')
    cmd = f"fslmerge -t {merged_nifti} {' '.join(niftis)}"
    os.system(cmd)
    assert Path(merged_nifti).exists()

    return merged_nifti

def merge_metrics(func_giis):
    merged_func_gii = create_temp_file('.func.gii')
    cmd = f"wb_command -metric-merge {merged_func_gii} -metric {' -metric '.join(func_giis)}"
    os.system(cmd)
    assert Path(merged_func_gii).exists()

    return merged_func_gii

NIFTIs - PALM

In [3]:
# Inputs
oscprep_base_dir = "/data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless"
experiment_id = '00_experiment-min'
data_type = 'CIFTI'
sub_id = '002'
ses_id = '02'
frequencies = [.125,.2]
out_dir = f"/data/fastfmri_toolbox_test_data/3T/bids/derivatives/TEST_PALM_{data_type}/sub-{sub_id}_ses-{ses_id}"
surfaces = {
    'cortex_left': "/data/dump/surfaces/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii",
    'cortex_right': "/data/dump/surfaces/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii",
}

test_flag = False

temp_files = [] # Track tmp files
for ix, _frequency in enumerate(frequencies):
    
    if ix > 0 and test_flag:
        break

    # Grab all ciftis for one-sample t-test
    ciftis, n_runs = {}, {}
    for task_id in ['entrain', 'control']:
        _ciftis = !ls {fla_dir}/{experiment_id}/{data_type}/sub-{sub_id}/ses-{ses_id}/task-{task_id}*/run-??/GLM/frequency*{_frequency}_z_score.dscalar.nii
        _ciftis.sort()
        ciftis[task_id] = _ciftis
        n_runs[task_id] = len(ciftis[task_id])

    # Create design matrix (.csv)
    design_matrix = np.zeros((sum(n_runs.values()), 2))
    idx1, idx2 = 0, 0
    for ix, _n_runs in enumerate(n_runs.values()):
        if ix > 0:
            idx1 += _n_runs
        idx2 += _n_runs
        design_matrix[idx1:idx2, ix] = 1        
    dm_csv = create_temp_file('.csv')
    with open(dm_csv, 'w') as f:
        for ix, i in enumerate(range(design_matrix.shape[0])):
            f.write(','.join([str(j) for j in design_matrix[i,:]]))
            if ix == design_matrix.shape[0] - 1:
                continue
            f.write('\n')
    temp_files.append(dm_csv) # Track tmp files

    # Create contrast (.csv)
    contrast_matrix = np.zeros((3,2))
    # Two sample t-test [entrain>control]
    contrast_matrix[0,0] = 1
    contrast_matrix[0,1] = -1
    contrast_matrix[1,0] = 1 # one sample t-test [entrain]
    contrast_matrix[2,1] = 1 # one sample t-test [control]
    contrast_csv = create_temp_file('.csv')
    with open(contrast_csv, 'w') as f:
        for ix,i in enumerate(range(contrast_matrix.shape[0])):
            f.write(','.join([str(j) for j in contrast_matrix[i,:]]))
            if ix == contrast_matrix.shape[0] - 1:
                continue
            f.write('\n')
    temp_files.append(contrast_csv) # Track tmp files

    # Concatenate all niftis across task conditions into a single list
    # Note: they must be ordered according to the design matrix
    listify_ciftis = ciftis['entrain'] + ciftis['control']

    # Separate ciftis
    listify_sep_ciftis = {}
    for ix, dscalar in enumerate(listify_ciftis):
        separated_dscalar = cifti_separate(dscalar)
        for k, v in separated_dscalar.items():
            if ix == 0:
                listify_sep_ciftis[k] = [v]
            else:
                listify_sep_ciftis[k].append(v)
            temp_files.append(v)

    # Merge separated ciftis
    merged_sep_cifti = {}
    merged_sep_cifti['volume'] = merge_niftis(listify_sep_ciftis['volume'])
    merged_sep_cifti['cortex_left'] = merge_metrics(listify_sep_ciftis['cortex_left'])
    merged_sep_cifti['cortex_right'] = merge_metrics(listify_sep_ciftis['cortex_right'])
    for k, v in merged_sep_cifti.items():
        temp_files.append(v)

    # Get surface and corresponding surface-vertex-area files
    surface_areas = get_surface_vertex_areas(surfaces['cortex_left'], surfaces['cortex_right'])
    for k, v in surface_areas.items():
        temp_files.append(v)

    # Verbose
    print(f'RUNNING: Frequency - {_frequency}')
    for _cifti in listify_ciftis:
        print(f"   - {_cifti}")

    # Run palm thrice, (1) volume, (2) CORTEX_LEFT, (3) CORTEX_RIGHT
    # Volume
    !/opt/PALM/run_palm.sh /opt/matlab \
        -i {merged_sep_cifti['volume']} \
        -d {dm_csv} \
        -t {contrast_csv} \
        -o {out_dir}/frequency-{str(_frequency).replace('.','')}_subcortical \
        -n 500 \
        -T -logp
    # Cortex Left
    !/opt/PALM/run_palm.sh /opt/matlab \
        -i {merged_sep_cifti['cortex_left']} \
        -s {surfaces['cortex_left']} {surface_areas['cortex_left']} \
        -d {dm_csv} \
        -t {contrast_csv} \
        -o {out_dir}/frequency-{str(_frequency).replace('.','')}_CortexLeft \
        -n 500 \
        -T -tfce2D -logp
    # Cortex Right
    !/opt/PALM/run_palm.sh /opt/matlab \
        -i {merged_sep_cifti['cortex_right']} \
        -s {surfaces['cortex_right']} {surface_areas['cortex_right']} \
        -d {dm_csv} \
        -t {contrast_csv} \
        -o {out_dir}/frequency-{str(_frequency).replace('.','')}_CortexRight \
        -n 500 \
        -T -tfce2D -logp
    
# Remove all contrast files
for f in temp_files:
    os.remove(f)

RUNNING: Frequency - 0.125
   - /data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla/00_experiment-min/CIFTI/sub-002/ses-02/task-entrainQ1/run-01/GLM/frequency-0.125_z_score.dscalar.nii
   - /data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla/00_experiment-min/CIFTI/sub-002/ses-02/task-entrainQ1/run-02/GLM/frequency-0.125_z_score.dscalar.nii
   - /data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla/00_experiment-min/CIFTI/sub-002/ses-02/task-entrainQ1/run-03/GLM/frequency-0.125_z_score.dscalar.nii
   - /data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla/00_experiment-min/CIFTI/sub-002/ses-02/task-entrainQ1/run-04/GLM/frequency-0.125_z_score.dscalar.nii
   - /data/fastfmri_toolbox_test_data/3T/bids/derivatives/oscprep_grayords_fmapless_fla/00_experiment-min/CIFTI/sub-002/ses-02/task-entrainQ1/run-05/GLM/frequency-0.125_z_score.dscalar.nii
   - /data/fastfmri_toolbox_

In [ ]:
!chmod -R 777 {out_dir}

In [ ]:
!rm -rf {out_dir} /tmp/*